In [ ]:
function [t_FTDP, f_FTDP ,Fcfo, FTDP_Window_Doppler, FTDP_Window_Doppler_Unaligned] = ProcessRawDataToAlignedDopplerProfile(fc,fs,fsc,doppler_window_size,win_adv,win_duration,display_option_1,display_option_2, display_option_3, dirpath, thefilename)
# Settings
f0 = fc
Fs = fs
Fsc = fsc
FTDP_adv = win_adv
winddur = win_duration
dfwin = doppler_window_size

# Will display the Aligned Doppler Spectrum for all processed windows in real-time
displayDopplerProfile_in_RealTime = display_option_1

# Will display the aligned Doppler Spectra (different from Doppler Profile)
# in real-time.
displayDopplerSpectra_in_RealTime = display_option_2

# Will compute and display the Unaligned Doppler Profile
showUnalignedDopplerProfile = display_option_3

dirpath0 = dirpath
fname = thefilename

## Define Constants
lambda = physconst('LightSpeed')/f0;

## Pre-Determine the frequency axis for the Doppler Profile (based on the window duration)
fsc_view = Fsc # Hz, Center Frequency for Frequency-Time Doppler Profile (FTDP)
wind = floor(winddur.*Fs) # samples, (or nfft size) this should be the desired window
FTDP_adv_samps = floor(FTDP_adv.*Fs) # samples, sliding window advance amount

f_FTDP = []; t_FTDP = 0;
f = ((0:wind-1)*(Fs/wind))';
f = f(f<Fs/2);
%f = ((-wind/2:wind/2 -1)*Fs/wind)'; % both sides of FFT

% Aligned Indexing
f0 = fsc_view;
th = 0; myStep = 1;
while 1
    if (length(find(abs(f-(f0-dfwin)) <= th)) == 1) && (length(find(abs(f-f0) <= th)) == 1) && (length(find(abs(f-(f0+dfwin-myStep)) <= th)) == 1)
        break;
    end
    myStep = myStep + 1;
    %th = th + 1;
end

scindex_lower = find(abs(f-(f0-dfwin)) <= th);
scindex_center = find(abs(f-f0) <= th);
scindex_upper = find(abs(f-(f0+dfwin-myStep)) <= th);
f_FTDP = f(scindex_lower:scindex_upper)-f0; % Get FTDP frequency range

% Unalgined Indexing
dfwin_unaligned = 40e3;
th = 1; myStep = 1;
while 0
    if (length(find(abs(f-(f0-dfwin_unaligned)) <= th)) == 1) && (length(find(abs(f-f0) <= th)) == 1) && (length(find(abs(f-(f0+dfwin_unaligned-myStep)) <= th)) == 1)
        break;
    end
    myStep = myStep + 1;
end

scindex_lower_unaligned = find(abs(f-(f0-dfwin_unaligned)) <= th);
scindex_center_unaligned = find(abs(f-f0) <= th);
scindex_upper_unaligned = find(abs(f-(f0+dfwin_unaligned-myStep)) <= th);
f_FTDP_unaligned = f(scindex_lower_unaligned:scindex_upper_unaligned)-f0; % Get FTDP frequency range

%% Doppler Processing
Fcfo = []; % Holder for CFO Measurement
FTDP_Window_Doppler_Unaligned = []; % Holder for FTDP Unaligned Doppler
FTDP_Window_Doppler = []; % Holder for FTDP Aligned Doppler
FTDP_Window_ind = 1; % Value for Window Index
nend = 1000000; % Controls how many windows are processed. Set to large number (1000000) as infinity
noPeaksPreviously = 0; % Control variable in case the FFT does not reveal a strong peak

x = read_complex_binary ([dirpath0 fname '.dat'],150e6); % Reads the complex-binary data

L = length(x);
for currSlideLoc = 0:FTDP_adv_samps:L-wind
    
    if  FTDP_Window_ind > nend
        break;
    end
    
    % Get window location and update t_FTDP (time vector)
    currTime = 0;
    if currSlideLoc == 0
       % For first segment (window) of signal
       windLoc = 1:wind;
    else
       % For all other segments of signal
       windLoc = currSlideLoc:currSlideLoc+wind-1;
       t_FTDP = [t_FTDP; t_FTDP(end)+FTDP_adv];
       currTime = t_FTDP(end)+FTDP_adv;
    end

    % Take large FFT over 1st window in this segment
    segment = x(windLoc);
    Fx = fft(segment);
    MFxPos = abs(Fx(1:wind/2));

    if showUnalignedDopplerProfile
        Fx22 = fft(hann(wind).*segment,wind);
        MFxPos22 = abs(Fx22(1:wind/2));
        FTDP_Window_Doppler_Unaligned(FTDP_Window_ind,:,1) = MFxPos22(scindex_lower_unaligned:scindex_upper_unaligned);
    end
    
    % Be sure only ONE massive peak exists (stronger in LOS than NLOS) if
    % not then need to skip over this signal segment. Time vector still
    % increases. For now it duplicates the window.
    
    % findLocs may need to be tunned for the specific Doppler profile
    % depending on the test equipment used. See findLocs.m, and edit the
    % parameters: 'MINPEAKHEIGHT',m,'MINPEAKDISTANCE',10e3, also notice it
    % is the locally normalized version of the current Doppler Spectra being processed, 
    % not the globally normalized version across the entire Doppler Profile. 
    locs = findLocs(MFxPos);
    if isempty(locs)
       display('Flag1: Empty locs')
       noPeaksPreviously = 1;
       FTDP_Window_ind = FTDP_Window_ind + 1;
       if FTDP_Window_ind ~= 1
        FTDP_Window_Doppler(FTDP_Window_ind,:) = FTDP_Window_Doppler(FTDP_Window_ind-1,:);
       end
       continue
    end

    locs = locs(end);
    if length(locs) ~= 1
       display('Flag2: locs length not one?')
       plot(f,MFxPos);
       break;
    end
    
    % Correct for CFO
    CFOfine = f(locs(1))-Fsc;
    Fcfo = [Fcfo f(locs(1))-Fsc];
    segment = segment.*exp(-1i.*2.*pi.*CFOfine.*[1:length(segment)]'.*(1/Fs));
    
    % Obtain corrected spectrum, which is the Doppler Profile for the
    % current window.
    Fx = fft(hann(wind).*segment);
    MFxPos = abs(Fx(1:wind/2));

    locs = findLocs(MFxPos);
    locs = locs(end);
    if length(locs) ~= 1
       display('Flag3: 2nd locs length not one?')
       break;
    end
    
    if noPeaksPreviously
        FTDP_Window_Doppler(FTDP_Window_ind,:) = MFxPos(scindex_lower:scindex_upper);
    else
        % Need to code: Estimate Doppler Profiles between noPeaks segement 
        % and current segment. For now just treat as normal.
        
        % <code to estimate missed segments>
        
        FTDP_Window_Doppler(FTDP_Window_ind,:) = MFxPos(scindex_lower:scindex_upper);
    end
       
    if displayDopplerProfile_in_RealTime
        figure(1)
        imagesc(156250-f(scindex_lower:scindex_upper),t_FTDP,20*log10(FTDP_Window_Doppler));
        title(fname)
        xlabel('Doppler Frequency (Hz)')
        ylabel('Time (s)')
        title(['WaterFall Plot: Window #' num2str(FTDP_Window_ind)])
        %axis([min(t_FTDP) max(t_FTDP) min(f_FTDP) max(f_FTDP)])
        colorbar
        drawnow
    end

    if displayDopplerSpectra_in_RealTime
        figure(2)
        plot(f_FTDP,flipud(MFxPos(scindex_lower:scindex_upper)))
        xlabel('Doppler Frequency (Hz)')
        ylabel('Energy (|Y|)')
        title(['Doppler Spectra for Window #' num2str(FTDP_Window_ind)])
        drawnow
    end
    
    % Update the FTDP window index and report status
    FTDP_Window_ind = FTDP_Window_ind +1;
    clc
    display('PRE-PROCESSING...')
    fprintf('Working %s: %.2f%% Complete\r',fname,100*(currSlideLoc/(L-wind)))
end

%% Show Aligned Doppler Profile
figure(3)
[rr cc ll] = size(FTDP_Window_Doppler);
typ = FTDP_Window_Doppler(1:rr,:);
if mean(f_FTDP) < 0
    typ = fliplr(FTDP_Window_Doppler(1:rr,:));
end
ampmin=max(max(abs(typ.')))/2000;
imagesc(t_FTDP(1:rr),f_FTDP,20*log10(max(abs(typ.'),ampmin)/ampmin));
title(['Doppler Profile: ' fname],'Interpreter','none')
ylabel('Doppler Frequency (Hz)')
xlabel('Time (s)')
NumTicks = 4;
L = get(gca,'XLim');
set(gca,'XTick',round(linspace(t_FTDP(1),t_FTDP(end),NumTicks)))
axis('xy')
colorbar

%% Show Unaligned Doppler Profile
if showUnalignedDopplerProfile
    figure
    typ = FTDP_Window_Doppler_Unaligned(:,:,1);
    if mean(f_FTDP_unaligned) < 0
        typ = fliplr(FTDP_Window_Doppler_Unaligned(:,:,1));
    end
    ampmin=max(max(abs(typ.')))/1000;
    imagesc(t_FTDP,0+f_FTDP_unaligned,flipud(20*log10(max(abs(typ.'),ampmin)/ampmin)));
    title('Doppler Domain Profile Unaligned, Fsc = 156250Hz')
    ylabel('Doppler Frequency (Hz)')
    xlabel('Time (s)')
    axis('xy')
    axis([min(t_FTDP) max(t_FTDP) min(0+f_FTDP_unaligned) max(0+f_FTDP_unaligned)])
    colorbar
end